### Install necessary packages

In [ ]:
# for web scraping
import requests
from bs4 import BeautifulSoup
import re
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import pandas as pd

In [ ]:
# for cleaning the list of key words
def clean(sentence_ws, sentence_pos):
    short_with_pos = []
    short_sentence = []
    
    # stop_pos means these categories are excluded
    stop_pos = set(['Nep', 'Nh', 'Nb', 'Neu', 'Nc', 'Nd'])
    
    for word_ws, word_pos in zip(sentence_ws, sentence_pos):
        # only keep N & V
        is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
        # delete words in stop_pos
        is_not_stop_pos = word_pos not in stop_pos
        # delete only one word
        is_not_one_charactor = not (len(word_ws) == 1)
        
        if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
            short_with_pos.append(f"{word_ws}({word_pos})")
            short_sentence.append(f"{word_ws}")
    return (" ".join(short_sentence), " ".join(short_with_pos))

### Set the target url and Scrape the text (using CKIP)

In [ ]:
url = "https://www.mbishop.com.tw/Article/Detail/79781"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')

pattern = re.compile(r"[\u4e00-\u9fff！？。，、]+")
raw_text = re.findall(pattern, soup.find_all("script")[25].get_text())
text = ["，".join([s for s in raw_text if s not in ["微軟正黑體"]])]

In [ ]:
# set CKIP Drivers
ws_driver  = CkipWordSegmenter(model="albert-base")
pos_driver = CkipPosTagger(model="albert-base")
ner_driver = CkipNerChunker(model="albert-base")

In [ ]:
# apply CKIP
ws  = ws_driver(text)
pos = pos_driver(ws)
ner = ner_driver(text)

In [ ]:
keyword_list = []
for sentence, sentence_ws, sentence_pos, sentence_ner in zip(text, ws, pos, ner):
    (short, res) = clean(sentence_ws, sentence_pos)
    keyword_list.extend(short.split(' '))

In [ ]:
# count key words in the whole 'keyword_list'
value_counts = pd.Series(keyword_list).value_counts()
df = pd.DataFrame({'Word': value_counts.index, 'Count': value_counts.values})
sub_df = df.sort_values(by=['Count'], ascending=False).reset_index(drop=True).head(100)

### Upload the dataframe to Google Sheet

In [ ]:
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

In [ ]:
# google sheet authorization
scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']

# set your api key (json file) as the credential
credentials = Credentials.from_service_account_file('YourJsonFile.json',scopes=scopes)

gc = gspread.authorize(credentials)

# get the google sheet id from the url of the google sheet
spreadsheet_key = 'YourGoogleSheetID'

# use the key and id to open a sheet in the google sheet
sheet = gc.open_by_key(spreadsheet_key).worksheet("TheSheetName")

# upload df to the google sheet
set_with_dataframe(sheet, sub_df)